In [1]:
import psycopg2
import pandas as pd
import numpy as np
import torch
import nltk
from sklearn.model_selection import train_test_split

conn_string = ""
conn = psycopg2.connect(conn_string)
cur=conn.cursor()

cur.execute("SELECT * FROM capstone.cap_data;")
result = cur.fetchall()

In [2]:
columns = ["row_id","subject_id","hadm_id","intime","outtime","information",'readmission']
df = pd.DataFrame(result,columns = columns)
df.head()

,row_id,subject_id,hadm_id,intime,outtime,information,readmission
0,31792,25996,135999,2200-04-14 19:12:00,2200-04-16 18:11:00,[**2200-4-16**] 12:21 PM\n CT HEAD W/O CONTRAS...,0
1,55392,88857,136000,2109-12-05 21:48:10,2109-12-06 16:16:49,Chief Complaint: ETOH withdrawal\n I saw and...,0
2,5401,4461,136001,2127-02-02 20:52:14,2127-03-02 15:16:49,[**2127-2-4**] 5:38 AM\n CHEST (PORTABLE AP) ...,0
3,13893,11342,136006,2176-06-04 19:13:08,2176-06-06 22:21:21,NURSING MICU NOTE 7P-7A\n\n46Y/O FEMALE PRESEN...,0
4,7825,6428,136010,2116-05-27 10:47:22,2116-05-29 15:35:55,[**2116-5-29**] 1:42 PM\n [**Last Name (un) 26...,0


In [3]:
import re

def preprocess(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    # remove, digits, spaces
    result = " ".join(y.split())
    
    return result

def make_str(lst):
    string = ""
    for sentence in lst:
        string += (sentence+"\n")
        
    return string

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['information'].values, df['readmission'].values, 
                                                    test_size=0.2, random_state=11)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  36753
Test set count:  9189


In [8]:
from transformers import BertTokenizerFast, BertForSequenceClassification 

tokenizer = BertTokenizerFast.from_pretrained('bvanaken/CORe-clinical-mortality-prediction')
model = BertForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
class mimicdataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = mimicdataset(train_input, y_train)
test_dataset = mimicdataset(test_input, y_test)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',          # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=2,              # 학습 에포크 수
    per_device_train_batch_size=8,   # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,   # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,                     # 학습할 모형
    args=training_args,              # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,     # 훈련 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정학습 실행
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)